# Nemotron Veritas: AI Agent for Deconstructing Misinformation

This notebook implements a sophisticated multi-agent system designed to act as a "critical thinking amplifier" using NVIDIA's Nemotron family of models. The system analyzes text to reveal rhetorical devices, logical fallacies, and persuasive techniques.

## System Architecture

1. **Architect Agent** (nvidia-nemotron-nano-9b-v2)
   - Analyzes text structure
   - Extracts main thesis and supporting claims

2. **Rhetoric Agent** (llama-3_3-nemotron-super-49b-v1_5)
   - Examines claims for logical fallacies
   - Uses RAG pipeline with fallacies knowledge base

3. **Synthesizer Agent** (nvidia-nemotron-nano-9b-v2)
   - Combines analyses
   - Formats structured output

## 1. Environment Setup and Dependencies

First, we'll install and import all required dependencies. Run this cell to set up your environment:

In [38]:
# Install required packages
!pip install -q langchain langchain-community faiss-cpu openai python-dotenv tqdm sentence-transformers langchain-nvidia-ai-endpoints

# Import dependencies
import os
import json
import logging
from typing import List, Dict, Any
from dataclasses import dataclass
from tqdm import tqdm

from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings, NVIDIARerank

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

## 2. API Configuration and Model Initialization

Configure the NVIDIA API credentials and initialize our three Nemotron models. We'll create custom wrapper classes for each model to handle their specific requirements:

In [39]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Model Configuration
NANO_API_KEY = "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"
SUPER_API_KEY = "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"

# Initialize our models using langchain-nvidia-ai-endpoints
ARCHITECT_MODEL = ChatNVIDIA(
    api_key=NANO_API_KEY,
    model="nvidia/nvidia-nemotron-nano-9b-v2",
    temperature=0.3,
    top_p=0.95,
    max_tokens=2048
)

RHETORIC_MODEL = ChatNVIDIA(
    api_key=SUPER_API_KEY,
    model="nvidia/llama-3.3-nemotron-super-49b-v1.5",
    temperature=0.7,
    top_p=0.95,
    max_tokens=65536
)

EMBEDDING_MODEL = NVIDIAEmbeddings(
    api_key=SUPER_API_KEY,
    model="nvidia/llama-3.2-nv-embedqa-1b-v2",
    truncate="END"  # Handle long texts gracefully
)

RERANKER_MODEL = NVIDIARerank(
    api_key=SUPER_API_KEY,
    model="nvidia/llama-3.2-nv-rerankqa-1b-v2"
)

print("Models initialized successfully!")

C:\Users\gdwu0\AppData\Local\Temp\ipykernel_5128\1596275397.py:12: DeprecationWarning: The 'max_tokens' parameter is deprecated and will be removed in a future version. Please use 'max_completion_tokens' instead.
  ARCHITECT_MODEL = ChatNVIDIA(


Models initialized successfully!


c:\Users\gdwu0\nemotron-veritas-local\.venv\Lib\site-packages\langchain_nvidia_ai_endpoints\_common.py:229: UserWarning: Found nvidia/nvidia-nemotron-nano-9b-v2 in available_models, but type is unknown and inference may fail.
  warnings.warn(
C:\Users\gdwu0\AppData\Local\Temp\ipykernel_5128\1596275397.py:20: DeprecationWarning: The 'max_tokens' parameter is deprecated and will be removed in a future version. Please use 'max_completion_tokens' instead.
  RHETORIC_MODEL = ChatNVIDIA(


## 3. Knowledge Base Construction

Now we'll create our knowledge base of logical fallacies and rhetorical devices. We'll use FAISS for vector storage and nvidia-embed-qa-4 for embeddings:

In [40]:
# Define our logical fallacies database
LOGICAL_FALLACIES = [
    {
        "name": "Ad Hominem",
        "description": "Attacking the person instead of addressing their argument",
        "example": "You can't trust his economic policy because he's never had a real job."
    },
    {
        "name": "Straw Man",
        "description": "Misrepresenting someone's argument to make it easier to attack",
        "example": "Environmentalists want us to go back to living in caves without electricity."
    },
    # Add more fallacies here
]

# Create documents from fallacies
documents = []
for fallacy in LOGICAL_FALLACIES:
    text = f"Fallacy: {fallacy['name']}\nDescription: {fallacy['description']}\nExample: {fallacy['example']}"
    documents.append(Document(page_content=text, metadata=fallacy))

# Create FAISS vector store with embeddings
vectordb = FAISS.from_documents(documents, EMBEDDING_MODEL)

# Create a retriever with reranking for better results
base_retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 3})
fallacy_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=RERANKER_MODEL
)

print(f"Created vector store with {len(LOGICAL_FALLACIES)} fallacies")

Created vector store with 2 fallacies


## 4. Architect Agent Implementation

The Architect Agent analyzes the text structure to identify the main thesis and supporting claims:

In [41]:
@dataclass
class TextStructure:
    thesis: str
    claims: List[str]

class ArchitectAgent:
    def __init__(self, model: ChatNVIDIA):
        self.model = model
        self.prompt_template = """
        Analyze the following text and identify:
        1. The main thesis (central argument)
        2. The key supporting claims
        
        Text: {text}
        
        Output your analysis in JSON format with 'thesis' and 'claims' fields.
        """
    
    def analyze_structure(self, text: str) -> TextStructure:
        prompt = self.prompt_template.format(text=text)
        
        # Get response from model using ChatNVIDIA interface
        messages = [{"role": "system", "content": prompt}]
        response = self.model.invoke(messages).content
        
        # Parse JSON response
        try:
            result = json.loads(response)
            return TextStructure(
                thesis=result['thesis'],
                claims=result['claims']
            )
        except Exception as e:
            logging.error(f"Error parsing Architect agent response: {e}")
            raise

# Initialize Architect agent
architect = ArchitectAgent(ARCHITECT_MODEL)
print("Architect Agent initialized!")

Architect Agent initialized!


## 5. Rhetoric Agent Implementation

The Rhetoric Agent examines each claim for logical fallacies using RAG:

In [42]:
@dataclass
class FallacyAnalysis:
    claim: str
    detected_fallacies: List[Dict[str, Any]]
    explanation: str

class RhetoricAgent:
    def __init__(self, model: ChatNVIDIA, retriever: ContextualCompressionRetriever):
        self.model = model
        self.retriever = retriever
        self.prompt_template = """You are a logical fallacy detection system. Your task is to analyze the following claim for logical fallacies using the provided context.

Claim: {claim}

Relevant fallacy patterns for reference:
{context}

Analyze the claim and output your findings in the following JSON format EXACTLY:
{{
    "detected_fallacies": [
        {{"name": "Fallacy Name", "type": "Type of Fallacy"}}
    ],
    "explanation": "Detailed explanation of how these fallacies are used in the claim"
}}

Ensure your response is ONLY the JSON object, with no additional text before or after.
If no fallacies are found, return an empty list for detected_fallacies.
"""
    
    def analyze_claim(self, claim: str) -> FallacyAnalysis:
        # Get relevant fallacy patterns with reranking
        docs = self.retriever.invoke(claim)  # Using new invoke method
        context = "\n\n".join(doc.page_content for doc in docs)
        
        # Generate analysis
        messages = [{"role": "system", "content": self.prompt_template.format(claim=claim, context=context)}]
        response = self.model.invoke(messages).content.strip()
        
        try:
            # Try to fix common JSON formatting issues
            if not response.startswith('{'):
                # Find the first occurrence of '{'
                start_idx = response.find('{')
                if start_idx != -1:
                    response = response[start_idx:]
                    # Find the last occurrence of '}'
                    end_idx = response.rfind('}')
                    if end_idx != -1:
                        response = response[:end_idx + 1]
            
            result = json.loads(response)
            
            # Ensure the required fields exist
            if "detected_fallacies" not in result or "explanation" not in result:
                result = {
                    "detected_fallacies": [],
                    "explanation": "Failed to detect fallacies in a structured way."
                }
                
            return FallacyAnalysis(
                claim=claim,
                detected_fallacies=result['detected_fallacies'],
                explanation=result['explanation']
            )
        except Exception as e:
            logging.error(f"Error parsing Rhetoric agent response: {e}")
            logging.error(f"Raw response: {response}")
            # Return a safe fallback response
            return FallacyAnalysis(
                claim=claim,
                detected_fallacies=[],
                explanation="Failed to analyze this claim due to processing error."
            )

# Initialize Rhetoric agent
rhetoric = RhetoricAgent(RHETORIC_MODEL, fallacy_retriever)
print("Rhetoric Agent initialized!")

Rhetoric Agent initialized!


## 6. Synthesizer Agent Implementation

The Synthesizer Agent combines analyses into a structured report:

In [43]:
@dataclass
class AnalysisReport:
    text: str
    thesis: str
    claims_analysis: List[FallacyAnalysis]
    summary: str

class SynthesizerAgent:
    def __init__(self, model: ChatNVIDIA):
        self.model = model
        self.prompt_template = """
        Create a comprehensive summary of the rhetorical analysis:
        
        Original Text: {text}
        
        Main Thesis: {thesis}
        
        Claims Analysis:
        {claims_analysis}
        
        Generate a clear, concise summary that explains how rhetorical devices and logical fallacies are used in this text.
        Focus on patterns and overall persuasive strategy.
        """
    
    def create_report(self, text: str, structure: TextStructure, 
                     claims_analysis: List[FallacyAnalysis]) -> AnalysisReport:
        # Format claims analysis for prompt
        claims_str = "\n\n".join(
            f"Claim: {analysis.claim}\n"
            f"Fallacies: {', '.join(f['name'] for f in analysis.detected_fallacies)}\n"
            f"Explanation: {analysis.explanation}"
            for analysis in claims_analysis
        )
        
        # Generate summary using ChatNVIDIA interface
        prompt = self.prompt_template.format(
            text=text,
            thesis=structure.thesis,
            claims_analysis=claims_str
        )
        messages = [{"role": "system", "content": prompt}]
        summary = self.model.invoke(messages).content
        
        return AnalysisReport(
            text=text,
            thesis=structure.thesis,
            claims_analysis=claims_analysis,
            summary=summary
        )

# Initialize Synthesizer agent
synthesizer = SynthesizerAgent(ARCHITECT_MODEL)  # Using nano-9b-v2 for formatting
print("Synthesizer Agent initialized!")

Synthesizer Agent initialized!


## 7. Orchestration Logic

Now we'll implement the main orchestration loop that coordinates our three agents:

In [44]:
class VeritasOrchestrator:
    def __init__(self):
        self.architect = architect
        self.rhetoric = rhetoric
        self.synthesizer = synthesizer
    
    def analyze_text(self, text: str) -> AnalysisReport:
        try:
            # Step 1: Extract structure
            logging.info("Analyzing text structure...")
            structure = self.architect.analyze_structure(text)
            
            # Step 2: Analyze each claim
            logging.info("Analyzing claims for fallacies...")
            claims_analysis = []
            for claim in structure.claims:
                analysis = self.rhetoric.analyze_claim(claim)
                claims_analysis.append(analysis)
            
            # Step 3: Generate final report
            logging.info("Generating final report...")
            report = self.synthesizer.create_report(
                text=text,
                structure=structure,
                claims_analysis=claims_analysis
            )
            
            return report
            
        except Exception as e:
            logging.error(f"Error in analysis pipeline: {e}")
            raise

# Initialize orchestrator
veritas = VeritasOrchestrator()
print("Veritas system initialized and ready for analysis!")

Veritas system initialized and ready for analysis!


# Testing Suite

Let's test our system with a variety of challenging texts that exhibit different types of misinformation and rhetorical manipulation:

1. **Emotional Appeal + Fear Mongering**
2. **False Authority + Cherry Picking**
3. **Conspiracy Theory Structure**
4. **False Dichotomy + Slippery Slope**
5. **Appeal to Nature + False Causation**

In [ ]:
# Test Case 1: Emotional Appeal + Fear Mongering (Climate Change Denial)
test_text_1 = """
Think about your children's future! The so-called 'green energy transition' is nothing but a scam that will destroy our economy 
and leave millions of hardworking Americans jobless. These climate alarmists want you to believe that every storm is a sign of 
impending doom, while they fly around in private jets. How many more families need to suffer under crushing energy prices before 
we wake up? If we don't stop this radical agenda now, our entire way of life will collapse, and China will dominate the world 
economy. The real science, which the mainstream media won't show you, proves that climate has always changed naturally.
"""

# Test Case 2: False Authority + Cherry Picking (Health Misinformation)
test_text_2 = """
Dr. James Mitchell, a renowned wellness consultant with over 20 years of experience in alternative healing, has discovered what 
Big Pharma doesn't want you to know. In his groundbreaking research, which has been viewed by millions online, he found that 
common vaccines contain dangerous levels of toxins. A survey of his 500 patients showed that 78% reported feeling better after 
stopping vaccinations. Leading medical intuitive Sarah Bennett also confirms these findings. The pharmaceutical industry has 
spent billions trying to suppress this information, but the truth is finally coming out.
"""

# Test Case 3: Conspiracy Theory Structure (Political Manipulation)
test_text_3 = """
Connect the dots, people! It's no coincidence that every major world event in the last decade has followed the exact same 
pattern. A small group of elite globalists, working through their puppet organizations, orchestrate these crises to consolidate 
power. Just look at who benefits financially - always the same players. They control the media, the banks, and even the 
weather through secret technology. Anyone who questions this narrative is immediately silenced or discredited. Wake up and see 
the bigger picture before it's too late!
"""

# Test Case 4: False Dichotomy + Slippery Slope (Education Policy)
test_text_4 = """
There are only two choices in this critical debate about education: either we maintain strict traditional values in our schools, 
or we surrender our children to radical social experimenters. If we allow any changes to the current curriculum, it will start 
a devastating chain reaction. First, they'll remove classic literature, then they'll ban patriotic history, and before you know 
it, our children won't even learn basic math or science anymore. We must take a stand now - you're either with us or against 
our children's future.
"""

# Test Case 5: Appeal to Nature + False Causation (Health Products)
test_text_5 = """
Nature provided us with everything we need for perfect health - no artificial chemicals required! Since the rise of synthetic 
medications in the 1950s, we've seen a dramatic increase in chronic diseases. This isn't a coincidence. Our ancestors lived 
disease-free lives using only natural remedies. Our premium organic wellness crystals, sourced from ancient healing sites, 
harness the earth's natural energy fields. Studies show that people who reject artificial medicines live happier lives. Isn't 
it time you returned to nature's way?
"""

# Function to analyze multiple texts
def run_analysis_suite(texts):
    for i, text in enumerate(texts, 1):
        print(f"\n{'='*20} Test Case {i} {'='*20}\n")
        try:
            report = veritas.analyze_text(text)
            print(f"Main Thesis:\n{report.thesis}\n")
            print("Claims Analysis:")
            for analysis in report.claims_analysis:
                print(f"\nClaim: {analysis.claim}")
                print("Detected Fallacies:")
                for fallacy in analysis.detected_fallacies:
                    print(f"- {fallacy['name']}")
                print(f"Explanation: {analysis.explanation}")
            print(f"\nSummary:\n{report.summary}")
        except Exception as e:
            print(f"Error analyzing test case {i}: {str(e)}")

# Run all test cases
test_texts = [test_text_1, test_text_2, test_text_3, test_text_4, test_text_5]
run_analysis_suite(test_texts)

2025-10-13 12:25:37,750 - INFO - Analyzing text structure...
2025-10-13 12:25:41,993 - INFO - Analyzing claims for fallacies...
2025-10-13 12:25:41,993 - INFO - Analyzing claims for fallacies...
2025-10-13 12:26:03,146 - INFO - Generating final report...
2025-10-13 12:26:03,146 - INFO - Generating final report...


=== Nemotron Veritas Analysis Report ===

Main Thesis:
Tech giants are destroying society by making teenagers anti-social through excessive social media use, and their influence is harmful to children's future.

Claims Analysis:

Claim: Teenagers spend excessive hours on social media, leading them to become 'anti-social zombies'.
Detected Fallacies:
Explanation: The claim does not misrepresent an opposing argument (Straw Man) nor does it attack a person's character instead of addressing their argument (Ad Hominem). While the term 'anti-social zombies' uses hyperbolic language, it serves as a conclusion about the perceived effects of social media rather than a direct fallacy against a specific argument or individual. No fallacies from the provided patterns are present in the claim.

Claim: Experts who downplay the harms of screen time are biased, as they are allegedly paid by Big Tech.
Detected Fallacies:
- Ad Hominem
Explanation: The claim attacks the character and potential bias of th